#  The LightCurve class

##  Background
#### What kind of physical data are we representing and what do these quantities mean?

Astrophysical variable sources including transients like supernovae are sources that are (roughly) fixed in position but change appreciably at human time scales, such that their measurements at different points of time in a survey are expected to be different. The key science of such sources has to do with interpreting such data.

In a photometric or imaging survey, the raw data associated with such measurements is in the form of a time series of images, along with metadata. This is often extracted into a catalog summarizing the inferred brightness of the source in the filter band and the uncertainties of the inferred. The uncertainties in this process are dominated by the sky noise which is Poisson distributed to a good approximation. Hence, the format of this recorded data is usually of the form

`timestamp`, `flux`, `flux uncertainty`, `zero point`, `magnitude system`

Note that the `flux` is really the band flux obtained in a filter band

The `zero point`, and `magnitude system` are methods of specifying the units of the reported `flux`, and `flux uncertainty`. `flux` may be reported so that the following equation with `bandflux` and `std_bandflux` in the same physical units holds. The `std_bandflux` depends on the transmissions and the magnitude system used.

bandmag := -2.5 log10(bandflux / std_bandflux) = -2.5 log10(flux) + zero point

so that a zero point of zero, implies that `flux` is `bandflux` in units of `std_bandflux`

## Desired Functionality

the `LightCurve` class is meant to be a standard internal representation of this kind of catalog data in our software. This means that 
- given a different format/model of data, we will first convert it into an instance of this class.
- All our calculations will be defined on instances of this class
- We should have output methods to convert the instances of this class to other useful output formats

## Structure

In [1]:
import sncosmo
import analyzeSN as ans

In [2]:
from analyzeSN import LightCurve

#### Instantiation

The class is instantiated using a constructor that has a mandatory input of a dataframe. and optional inputs. The dataframe must contain the light curve information, and the minimum set of columns required are:
`mjd`, `band`, `flux`, `flux_err`, `zp`, `zpsys` or recognized aliases. Here, we load some data and demonstrate this

In [3]:
ex_data = sncosmo.load_example_data().to_pandas()
ex_data.head()

,time,band,flux,fluxerr,zp,zpsys
0,55070.000000,sdssg,0.363512,0.672844,25.0,ab
1,55072.051282,sdssr,-0.200801,0.672844,25.0,ab
2,55074.102564,sdssi,0.307494,0.672844,25.0,ab
3,55076.153846,sdssz,1.087761,0.672844,25.0,ab
4,55078.205128,sdssg,-0.436679,0.672844,25.0,ab


This was a SNCosmo example dataset, loaded into `pandas`. Note that the column representing temporal information is `time`. Now, we will use this to instantiate our class.

In [4]:
lc = LightCurve(ex_data)

Now, we can see the light curve by using the attribute `lightCurve` which outputs a dataFrame in our convention. We note that `time` has changed to our standard name `mjd`

In [5]:
lc.lightCurve.head()

,mjd,band,flux,fluxerr,zp,zpsys
0,55070.000000,sdssg,0.363512,0.672844,25.0,ab
1,55072.051282,sdssr,-0.200801,0.672844,25.0,ab
2,55074.102564,sdssi,0.307494,0.672844,25.0,ab
3,55076.153846,sdssz,1.087761,0.672844,25.0,ab
4,55078.205128,sdssg,-0.436679,0.672844,25.0,ab


What would have happened if an essential column was missing? to check let us do the following:
   - We create a dataframe like the above, but remove the essential column band
   - We attempt to create an instance of the LightCurve class

In [6]:
lc_tmp = ex_data.copy()
del lc_tmp['band']
lc_tmp.head()

,time,flux,fluxerr,zp,zpsys
0,55070.000000,0.363512,0.672844,25.0,ab
1,55072.051282,-0.200801,0.672844,25.0,ab
2,55074.102564,0.307494,0.672844,25.0,ab
3,55076.153846,1.087761,0.672844,25.0,ab
4,55078.205128,-0.436679,0.672844,25.0,ab


In [7]:
lc_bad = LightCurve(lc_tmp)

ValueError: ('light curve data has missing columns', set(['band']))

This raises a ValueError pointing out that the column `band` was missing. However, it correctly recognizes that while
`mjd` was missing, it had the aliased column `time` instead and did not report it missing.

To get back the SNCosmo light curve data

In [8]:
SNCosmoLC = lc.snCosmoLC()

In [9]:
SNCosmoLC

time,band,flux,fluxerr,zp,zpsys
float64,str5,float64,float64,float64,str2
55070.0,sdssg,0.36351153597,0.672843847541,25.0,ab
55072.0512821,sdssr,-0.200801295864,0.672843847541,25.0,ab
55074.1025641,sdssi,0.307494232981,0.672843847541,25.0,ab
55076.1538462,sdssz,1.08776103656,0.672843847541,25.0,ab
55078.2051282,sdssg,-0.43667895645,0.672843847541,25.0,ab
55080.2564103,sdssr,1.09780966779,0.672843847541,25.0,ab
55082.3076923,sdssi,3.7562685627,0.672843847541,25.0,ab
55084.3589744,sdssz,5.34858894966,0.672843847541,25.0,ab
55086.4102564,sdssg,2.82614187269,0.672843847541,25.0,ab


## SNANA simulations

In [10]:
from analyzeSN import SNANASims

In [11]:
from astropy.table import Table

In [12]:
snana_eg = SNANASims.fromSNANAfileroot(snanafileroot='snana_fits', location='../analyzeSN/example_data/',
                                       coerce_inds2int=False)

In [13]:
Table.read(snana_eg.photFile)[45:50]

MJD,FLT,FIELD,TELESCOPE,PHOTFLAG,PHOTPROB,FLUXCAL,FLUXCALERR,MAG,MAGERR,PSF_SIG1,PSF_SIG2,PSF_RATIO,SKY_SIG,SKY_SIG_T,RDNOISE,ZEROPT,ZEROPT_ERR,GAIN
float64,str2,str12,str20,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
53023.23,r,NULL,ALL,0,0.0,0.82,1.756,27.717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53026.27,i,NULL,ALL,0,0.0,2.54,2.877,26.487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53026.34,r,NULL,ALL,0,0.0,-3.1,1.816,666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-777.0,-,XXXX,XXXX,0,0.0,-777.0,-777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52881.5,i,NULL,ALL,0,0.0,-3.9,2.704,666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let us look at what the array in the fits file looks like: 
- It has fields that are not of the standard name (FLUXCAL)
- It has things like "XXX" in he fields

In [18]:
lcInstance = snana_eg.get_SNANA_photometry(snid='03D1aw')

In [20]:
lcInstance.lightCurve[['mjd', 'band', 'flux', 'fluxerr', 'zp', 'zpsys']]

,mjd,band,flux,fluxerr,zp,zpsys
0,52881.500,lsst_i,13.440000,2.846000,27.5,ab
1,52881.539,lsst_r,7.740000,1.701000,27.5,ab
2,52881.559,lsst_z,19.719999,6.539000,27.5,ab
3,52886.602,lsst_i,44.520000,2.909000,27.5,ab
4,52900.531,lsst_i,86.570000,2.914000,27.5,ab
5,52900.559,lsst_r,62.169998,2.324000,27.5,ab
6,52900.590,lsst_z,84.410004,8.752000,27.5,ab
7,52904.531,lsst_i,80.720001,2.710000,27.5,ab
8,52904.570,lsst_r,55.090000,2.009000,27.5,ab
9,52908.559,lsst_r,54.500000,1.930000,27.5,ab


In [16]:
lcInstance.bandNameDict

{'Y': 'lsst_y',
 'g': 'lsst_g',
 'i': 'lsst_i',
 'r': 'lsst_r',
 'u': 'lsst_u',
 'z': 'lsst_z'}

In [27]:
snana_eg.headData['REDSHIFT_FINAL']

SNID
03d1aw    0.5817
03d1ax    0.4960
Name: REDSHIFT_FINAL, dtype: float32

## Fitting

In [21]:
import sncosmo

In [33]:
model = sncosmo.Model(source='salt2-extended')

In [28]:
model.set(z=0.5817)

In [40]:
res = sncosmo.mcmc_lc(lcInstance.snCosmoLC(), model, vparam_names=['t0', 'x0', 'x1', 'c'], modelcov=False)

In [41]:
from analyzeSN import ResChar

In [44]:
reschar = ResChar.fromSNCosmoRes(res)

In [45]:
reschar.salt_samples()

,t0,x1,c,mB,mu
0,52897.907859,-3.000480,0.912710,13.531365,10.281897
1,52897.907859,-3.000480,0.912710,13.531365,10.281897
2,52897.950865,-3.026473,0.907221,13.519910,10.283819
3,52897.950865,-3.026473,0.907221,13.519910,10.283819
4,52897.950865,-3.026473,0.907221,13.519910,10.283819
5,52897.621743,-2.825333,0.950116,13.617200,10.276294
6,52897.542850,-2.727438,0.962402,13.637706,10.272418
7,52897.542850,-2.727438,0.962402,13.637706,10.272418
8,52897.417684,-2.647286,0.975002,13.662726,10.269600
9,52897.637988,-2.788180,0.945702,13.594616,10.272595


# Scratch

In [10]:
10**(-0.4 * 0.27)

0.7798301105232587

In [11]:
-2.5 * np.log10(f) 

NameError: name 'np' is not defined

In [3]:
10.0**(0.27 / 2.5 )

1.2823305826560216